In [1]:
from data_loading import *
from torch.utils.data import DataLoader

# One method to rule them all

In [3]:
train_dataset, test_dataset = get_dataset('gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr')

/usr/local/anaconda3/envs/mlg_310/lib/python3.10/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Model surface dataset size 75.83941642194986 GiB


## If you want to try out with custom arguments...

In [2]:
era5 = get_xarray_dataset('gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr')
era5 = era5[['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature']]
era5 = era5.sel(time=slice('2010-01-01', '2010-12-31'))
era5 = era5.to_array()

Model surface dataset size 75.83941642194986 GiB


/usr/local/anaconda3/envs/mlg_310/lib/python3.10/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [3]:
train_dataset, test_dataset = get_weather_dataset_from_array(era5, obs_window=4, pred_window=1, overlap=False, group_by_time=True, test_size=0.2)
train_dataset, test_dataset

Lag features created successfully
Train-test split done
Scaling done


(<data_loading.WeatherDataset at 0x7f7aa9efee00>,
 <data_loading.WeatherDataset at 0x7f7a92fa5b10>)

In [5]:
torch.save(train_dataset, 'datasets/train_dataset.pt')
torch.save(test_dataset, 'datasets/test_dataset.pt')

In [7]:
train_dataset = torch.load('datasets/train_dataset.pt')
train_dataset

In [4]:
# Parameters
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train model...

Now we train the model